In [133]:
import pandas as pd
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

# Load the data from the CSV file
data = pd.read_csv('ball_to_ball_data.csv')

print(data.columns)

Index(['match_id', 'date', 'venue', 'batting_team', 'bowling_team', 'innings',
       'over', 'striker', 'non_striker', 'bowler', 'Runs_scored',
       'runs_of_bat', 'extras', 'wide', 'legbyes', 'byes', 'noballs',
       'wicket_type', 'player_dismissed', 'fielder'],
      dtype='object')


In [130]:
# Create an empty DataFrame for the aggregated player statistics
columns = ['match_id', 'date', 'venue', 'player', 'team', 'runs_scored', 'balls_faced', 'fours', 'sixes', 'strike_rate', 'wickets_taken', 'runs_conceded', 'balls_bowled', 'no_dismissal', 'economy']
matchwise_player_stats = pd.DataFrame(columns=columns)

# Iterate through each match
for match_id in data['match_id'].unique():
    match_data = data[data['match_id'] == match_id]
    date = match_data['date'].iloc[0]
    venue = match_data['venue'].iloc[0]
    
    # Get all players involved in the match
    players = set(match_data['striker']).union(set(match_data['non_striker'])).union(set(match_data['bowler']))
    
    for player in players:
        player_data = {
            'match_id': match_id,
            'date': date,
            'venue': venue,
            'player': player,
            'team': '',
            'runs_scored': 0,
            'balls_faced': 0,
            'fours': 0,
            'sixes': 0,
            'strike_rate': 0.0,
            'wickets_taken': 0,
            'runs_conceded': 0,
            'balls_bowled': 0,
            'no_dismissal': 0,
            'economy': 0.0
        }
        
        # Batting stats
        batting_data = match_data[(match_data['striker'] == player) | (match_data['non_striker'] == player)]
        if not batting_data.empty:
            if not match_data[match_data['striker'] == player].empty:
                player_data['team'] = match_data[match_data['striker'] == player]['batting_team'].iloc[0]
            else:
                player_data['team'] = match_data[match_data['non_striker'] == player]['batting_team'].iloc[0]
            
            player_data['runs_scored'] = batting_data[batting_data['striker'] == player]['runs_of_bat'].sum()
            player_data['balls_faced'] = batting_data[(batting_data['striker'] == player) & (batting_data['wide'] == 0) & (batting_data['noballs'] == 0)].shape[0]
            player_data['fours'] = batting_data[(batting_data['striker'] == player) & (batting_data['runs_of_bat'] == 4)].shape[0]
            player_data['sixes'] = batting_data[(batting_data['striker'] == player) & (batting_data['runs_of_bat'] == 6)].shape[0]
            player_data['strike_rate'] = (player_data['runs_scored'] / player_data['balls_faced']) * 100 if player_data['balls_faced'] > 0 else 0
        
        # Bowling stats
        bowling_data = match_data[match_data['bowler'] == player]
        if not bowling_data.empty:
            player_data['team'] = bowling_data['bowling_team'].iloc[0]
            player_data['wickets_taken'] = bowling_data[bowling_data['wicket_type'].isin(['bowled', 'caught', 'lbw', 'stumped'])].shape[0]
            player_data['runs_conceded'] = bowling_data[(bowling_data['byes'] == 0) & (bowling_data['legbyes'] == 0)]['Runs_scored'].sum()
            player_data['balls_bowled'] = bowling_data[(bowling_data['wide'] == 0) & (bowling_data['noballs'] == 0)].shape[0]
            player_data['economy'] = (player_data['runs_conceded'] / player_data['balls_bowled']) * 6 if player_data['balls_bowled'] > 0 else 0
         
        # Dismissal stats
        player_data['no_dismissal'] = match_data[match_data['player_dismissed'] == player].shape[0]

       
        # Append the player data to the DataFrame
        matchwise_player_stats = pd.concat([matchwise_player_stats, pd.DataFrame([player_data])], ignore_index=True)

# Remove duplicates based on 'match_id' and 'player'
matchwise_player_stats = matchwise_player_stats.drop_duplicates(subset=['match_id', 'player'])

#handling values
matchwise_player_stats['economy'] = round(matchwise_player_stats['economy'], 2)
matchwise_player_stats['strike_rate'] = round(matchwise_player_stats['strike_rate'], 2)

#print(matchwise_player_stats.columns)
# Save the aggregated player statistics to a new CSV file
matchwise_player_stats.to_csv('matchwise_player_stats.csv', index=False)

C:\Users\DELL\AppData\Local\Temp\ipykernel_7968\4251641449.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  matchwise_player_stats = pd.concat([matchwise_player_stats, pd.DataFrame([player_data])], ignore_index=True)


In [131]:

# Load the player stats file
player_stats = pd.read_csv('matchwise_player_stats.csv')

# Create an empty DataFrame for the aggregated tournament player statistics
columns = ['player', 'team', 'number_of_matches_played', 'runs_scored', 'balls_faced', 'fours', 'sixes', 'strike_rate', 'wickets_taken', 'runs_conceded', 'balls_bowled', 'economy']
tournament_player_stats = pd.DataFrame(columns=columns)

# Group by player and team
grouped = player_stats.groupby(['player', 'team'])

# Compute aggregate statistics
tournament_player_stats['player'] = grouped['player'].first()  # The player name
tournament_player_stats['team'] = grouped['team'].first()  # The team name
tournament_player_stats['number_of_matches_played'] = grouped['match_id'].nunique()  # Number of unique matches played
tournament_player_stats['runs_scored'] = grouped['runs_scored'].sum()  # Total runs scored
tournament_player_stats['balls_faced'] = grouped['balls_faced'].sum()  # Total balls faced
tournament_player_stats['fours'] = grouped['fours'].sum()  # Total fours
tournament_player_stats['sixes'] = grouped['sixes'].sum()  # Total sixes
tournament_player_stats['strike_rate'] = tournament_player_stats.apply(lambda row: (row['runs_scored'] / row['balls_faced']) * 100 if row['balls_faced'] > 0 else 0, axis=1)  # Strike rate
tournament_player_stats['no_dismissal'] = grouped['no_dismissal'].sum()
tournament_player_stats['wickets_taken'] = grouped['wickets_taken'].sum()  # Total wickets taken
tournament_player_stats['runs_conceded'] = grouped['runs_conceded'].sum()  # Total runs conceded
tournament_player_stats['balls_bowled'] = grouped['balls_bowled'].sum()  # Total balls bowled
tournament_player_stats['economy'] = tournament_player_stats.apply(lambda row: (row['runs_conceded'] / row['balls_bowled']) * 6 if row['balls_bowled'] > 0 else 0, axis=1)  # Economy rate

#handling values
tournament_player_stats['economy'] = round(tournament_player_stats['economy'], 2)
tournament_player_stats['strike_rate'] = round(tournament_player_stats['strike_rate'], 2)

# Save the aggregated tournament player statistics to a new CSV file
tournament_player_stats.to_csv('tournament_player_stats.csv', index=False)


In [132]:
# Assuming 'data' is your DataFrame
data = pd.read_csv('ball_to_ball_data.csv')  # Replace this with your actual data loading method

# Filter matches up to match_id < 70
data_before_70 = data[data['match_id'] < 71]

# Calculate the total number of matches played by each team
matches_played = data_before_70.groupby('batting_team')['match_id'].nunique().reset_index(name='league_matches_played')

# Calculate the total runs scored by each team
runs_scored = data.groupby('batting_team')['Runs_scored'].sum().reset_index(name='total_runs_scored')

# Calculate the total wickets taken by each team
wickets = data[data['wicket_type'].notnull()]
wickets_taken = wickets.groupby('bowling_team')['player_dismissed'].count().reset_index(name='total_wickets_taken')

# Calculate wins, losses, and ties
# First, get the total runs scored in each match by each team
match_scores = data.groupby(['match_id', 'batting_team'])['Runs_scored'].sum().reset_index()

# Pivot the data to get team scores side by side for each match
match_scores_pivot = match_scores.pivot(index='match_id', columns='batting_team', values='Runs_scored').reset_index()


# Determine match results
match_results = []
for _, row in match_scores_pivot.iterrows():
    teams = row.drop('match_id').dropna().index.tolist()
    if len(teams) == 2:
        team1, team2 = teams
        score1, score2 = row[team1], row[team2]
        if score1 > score2:
            match_results.append({'match_id': row['match_id'], 'winner': team1, 'loser': team2, 'tie': False})
        elif score1 < score2:
            match_results.append({'match_id': row['match_id'], 'winner': team2, 'loser': team1, 'tie': False})
        else:
            match_results.append({'match_id': row['match_id'], 'winner': None, 'loser': None, 'tie': True})

match_results_df = pd.DataFrame(match_results)


# Calculate the number of wins, losses, and ties
wins = match_results_df['winner'].value_counts().reset_index(name='matches_won').rename(columns={'index': 'team'})
losses = match_results_df['loser'].value_counts().reset_index(name='matches_lost').rename(columns={'index': 'team'})
ties = match_results_df[match_results_df['tie']].shape[0]
teams = data_before_70['batting_team'].unique()

# Calculate ties for each team
matches_tied = {team: 14 - matches_played.set_index('batting_team').at[team, 'league_matches_played'] if team in matches_played['batting_team'].values else 14 for team in teams}
matches_tied_df = pd.DataFrame(list(matches_tied.items()), columns=['team', 'matches_tied'])

# Combine all the results into a single DataFrame
tournament_team_stats = pd.merge(matches_played, runs_scored, left_on='batting_team', right_on='batting_team')
tournament_team_stats = pd.merge(tournament_team_stats, wickets_taken, left_on='batting_team', right_on='bowling_team', how='left').drop(columns='bowling_team')
tournament_team_stats = pd.merge(tournament_team_stats, wins, left_on='batting_team', right_on='winner', how='left').drop(columns='winner').fillna({'matches_won': 0})
tournament_team_stats = pd.merge(tournament_team_stats, losses, left_on='batting_team', right_on='loser', how='left').drop(columns='loser').fillna({'matches_lost': 0})
tournament_team_stats = pd.merge(tournament_team_stats, matches_tied_df, left_on='batting_team', right_on='team', how='left').drop(columns='team')

# Rename columns for clarity
tournament_team_stats = tournament_team_stats.rename(columns={'batting_team': 'team'})

tournament_team_stats['total_matches_played'] = tournament_team_stats['matches_won'] + tournament_team_stats['matches_lost'] + tournament_team_stats['matches_tied']

tournament_team_stats['points'] = ((tournament_team_stats['matches_won'] - tournament_team_stats['matches_tied']) * 2) + (tournament_team_stats['matches_tied'] * 1)

tournament_team_stats = tournament_team_stats[['team','total_runs_scored', 'total_wickets_taken', 'league_matches_played', 'matches_won',  'matches_lost',  'matches_tied', 'points','total_matches_played']]

# Save the aggregated tournament team statistics to a new CSV file
tournament_team_stats.to_csv('tournament_team_stats.csv', index=False)

In [138]:
print(tournament_team_stats.columns)

Index(['team', 'total_runs_scored', 'total_wickets_taken',
       'league_matches_played', 'matches_won', 'matches_lost', 'matches_tied',
       'points', 'total_matches_played'],
      dtype='object')
